In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from NoiseKD import Teacher

First hurdle. the embedding layer.  It requires the ints as inputs, not one hot.  i need one hot.  i will also need to rewire the LLM i use to do the same.  hmm.  save the weights, load it into a model that has the same shapes, but accepts one hot arrays, not vocab_indexes.

In [3]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, hidden_dim, num_layers, dropout):
        super(TransformerEncoder, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout),
            num_layers
        )

    def forward(self, x):
        return self.transformer(x)

class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout):
        super(SimpleLanguageModel, self).__init__()

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Define the transformer encoder
        self.transformer_encoder = TransformerEncoder(embedding_dim, num_heads, hidden_dim, num_layers, dropout)

        # Define the output layer
        self.output_layer = nn.Linear(embedding_dim, class_num)

    def forward(self, input_data):
        # Input_data is of shape (batch_size, sequence_length)
        # Apply embedding layer
        #print(input_data.shape)
        embedded = self.embedding(input_data)
        #print(embedded.shape)
        # Pass through the transformer encoder
        transformed = self.transformer_encoder(embedded)

        # Average along the sequence dimension
        avg_transformed = torch.mean(transformed, dim=1)

        # Apply the output layer
        output = self.output_layer(avg_transformed)

        return torch.sigmoid(output)
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
embedding_dim = 16
num_heads = 8
hidden_dim  = 11
num_layers = 2
dropout = 0.1
vocab_size = 80
class_num = 14
batch_size = 23
sequence_length = 160

In [5]:
SLM = SimpleLanguageModel(vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout)

In [6]:
input_data = random_data = torch.randint(low=0, high=vocab_size, size=(batch_size, sequence_length))
input_data.shape,SLM(input_data).shape

(torch.Size([23, 160]), torch.Size([23, 14]))

In [7]:
count_parameters(SLM)

4580

In [8]:
teacher_slm = Teacher(SLM,(sequence_length,)) #don't specify batch!!

In [9]:
teacher_slm.configure(dist_type = "ints", gen_m = vocab_size, gen_n = 100, gen_epochs = 50,gen_lr = 0.01, out_type = "one-hot" ) #this is dying.  might be time for colab!!

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


Configuring Teacher:: 100%|█████████████████████| 50/50 [00:05<00:00,  9.78it/s]

Teacher Configured


In [10]:
args = { 'val_train' : "train"
                      , 'n' : 1000
                      , 'dist_type' : 'ints'
                      , 'm' : vocab_size
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

In [11]:
teacher_slm.model(teacher_slm.train_inputs).shape

torch.Size([1000, 14])

In [12]:
teacher_slm.train_inputs.shape

torch.Size([1000, 160])

In [13]:
teacher_slm.model.embedding.weight.detach().numpy()

array([[-0.6307676 , -1.5309151 ,  1.6552991 , ...,  0.5422383 ,
        -0.37892413,  0.03871037],
       [ 0.72592354, -0.2904947 ,  0.5345427 , ..., -0.23445489,
        -0.7751044 , -1.2644582 ],
       [-2.226684  , -1.8999157 ,  0.2106669 , ..., -0.6048749 ,
        -0.04418471,  0.7381425 ],
       ...,
       [-1.1524984 , -0.25857905, -0.60018116, ...,  0.09485833,
         0.5458435 ,  0.08286065],
       [-0.2966404 ,  0.77404463, -0.7438242 , ..., -0.48721117,
         0.94495773,  0.65280145],
       [-0.02648878,  0.2770553 ,  0.28099364, ..., -0.3222659 ,
         0.3414036 ,  0.08566129]], dtype=float32)

In [14]:
torch.argmax(teacher_slm.train_targets,axis = -1) #the code is write, i just need this to be a better dist.

tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 4,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 4, 9, 8, 9, 9, 9, 9, 9, 9, 8,
        9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 4, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9,
        9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9,
        8, 9, 9, 9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 4, 9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 4, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 8, 9,
        9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,

specify gen outputs.  we need one hots
i can do this better than chat gpt lol

In [15]:
##this function works, it is in LABNET.  
def one_hot_last_dim(tensor_shape):
    num_classes = tensor_shape[-1]
    random_idx = np.random.randint(1, num_classes + 1, size=tensor_shape[:-1])
    zero_tensor = np.zeros(tensor_shape, dtype=int)
    last_dim_indices = np.arange(num_classes)
    zero_tensor[..., :, last_dim_indices] = (random_idx[..., np.newaxis] == last_dim_indices)
    return zero_tensor